In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
2,COVID-19 concerns addressed KARE11.com,https://www.kare11.com/article/news/local/kare...,"Sat, 30 Jan 2021 13:58:28 GMT",2021-01-30 13:58:28+00:00
7,"Just As Vaccine Distribution Begins, New COVID...",https://www.npr.org/2021/01/30/962357968/just-...,"Sat, 30 Jan 2021 12:53:00 GMT",2021-01-30 12:53:00+00:00
1,One of largest cemeteries in U.S. struggles wi...,https://www.nbcnews.com/news/us-news/one-large...,"Sat, 30 Jan 2021 03:25:00 GMT",2021-01-30 03:25:00+00:00
9,Article: COVID Virus May Prompt Body to Attack...,https://www.webmd.com/lung/news/20210129/covid...,"Sat, 30 Jan 2021 00:38:57 GMT",2021-01-30 00:38:57+00:00
0,"Coronavirus (COVID-19) Update: January 29, 202...",https://www.fda.gov/news-events/press-announce...,"Fri, 29 Jan 2021 23:05:00 GMT",2021-01-29 23:05:00+00:00
8,Austin Public Health COVID-19 Vaccine Updates ...,https://austintexas.gov/news/austin-public-hea...,"Fri, 29 Jan 2021 22:40:02 GMT",2021-01-29 22:40:02+00:00
4,ClackCo Jan. 29 COVID-19 vaccine update: Three...,https://www.clackamas.us/news/2021-01-29/clack...,"Fri, 29 Jan 2021 21:56:15 GMT",2021-01-29 21:56:15+00:00
6,DHEC Launches New COVID-19 Vaccine Information...,https://scdhec.gov/news-releases/dhec-launches...,"Fri, 29 Jan 2021 17:12:20 GMT",2021-01-29 17:12:20+00:00
3,DHEC Confirms First Death from COVID-19-relate...,https://scdhec.gov/news-releases/dhec-confirms...,"Fri, 29 Jan 2021 15:54:16 GMT",2021-01-29 15:54:16+00:00
5,Janssen Investigational COVID-19 Vaccine: Inte...,https://www.nih.gov/news-events/news-releases/...,"Fri, 29 Jan 2021 12:04:08 GMT",2021-01-29 12:04:08+00:00
